<center> 

<h1 style="text-align:center"> System F </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

System F is polymorphic lambda calculus, i.e., STLC extended with polymorphic types. The syntax and semantics of System F are shown below (image taken from TAPL).

<table>
<tr>
<td><img src="images/SystemF.png"></td>
</tr>
</table>

STLC: \(x:int). x

System F: 
-- Id: \('a).\(x:'a). x

In [23]:
(* Abstract Syntax *)

(* System F type *)
type typ = 
  | TyVar of string
  | TyArr of typ * typ
  | TyPoly of string * typ (* \forall 'a. T *)
  
(* System F Expressions *)  
type expr = 
  | Var of string
  | Lam of string * typ * expr
  | App of expr * expr
  | TyLam of string * expr
  | TyApp of expr * typ

type typ = TyVar of string | TyArr of typ * typ | TyPoly of string * typ


type expr =
    Var of string
  | Lam of string * typ * expr
  | App of expr * expr
  | TyLam of string * expr
  | TyApp of expr * typ


In [24]:
(*
 let app (f : 'a. 'a -> 'a) (x:'b) : 'b = f x
*)
(*
 \'b. \(f: \'a. a' -> 'a). \(x:'b). (f ['b]) x
*)
TyLam("'b", Lam("f", TyPoly("'a",TyArr(TyVar "'a", TyVar "'a")), 
                Lam("x", TyVar "'b", 
                        App(TyApp(Var "f", TyVar "'b"), Var "x"))))

- : expr =
TyLam ("'b",
 Lam ("f", TyPoly ("'a", TyArr (TyVar "'a", TyVar "'a")),
  Lam ("x", TyVar "'b", App (TyApp (Var "f", TyVar "'b"), Var "x"))))


In [31]:
module SystemF = struct
    exception TypeError
    type typbind = string * typ
    type tyenv = typbind list


    (* [] |- \'a. \(x:'a). x : \'a. 'a -> 'a *)
    let rec well_formed (env:tyenv) typ : bool = match typ with
      | TyVar a -> List.exists 
                      (function ("",TyVar b) when a=b -> true
                              | _ -> false) env
      | TyArr(t1,t2) -> (well_formed env t1) && (well_formed env t2)
      | TyPoly(a,t) -> well_formed (("",TyVar a)::env) t

    (* old_subst : (expr,string) -> expr -> expr *) 
    (* [t/x]t1 --> t2 *)
    let rec subst ((t,x): (typ*string)) (t1:typ) : typ = match t1 with
      | TyVar y when y=x -> t
      | TyVar y -> t1
      | TyArr (t11,t12) -> TyArr (subst (t,x) t11 ,subst (t,x) t12)
      | TyPoly(y,t11) when (x=y) -> t1
      | TyPoly(y,t11) -> TyPoly(y, subst (t,x) t11)


    let rec type_check (env: tyenv) (e: expr) : typ = match e with
      | Var x -> (try List.assoc x env with Not_found -> raise TypeError)
      | Lam(x,ty,e1) when (well_formed env ty) -> 
        let bdy_typ = type_check ((x,ty)::env) e1 in
        TyArr(ty,bdy_typ)
      | Lam(_,_,_) -> raise TypeError
      | App(e1,e2) -> 
        let ty1 = type_check env e1 in
        let ty2 = type_check env e2 in
        begin
          match ty1 with
          | TyArr (t1,t2) when (t1 = ty2) -> t2
          | _ -> raise TypeError
        end
      | TyLam(a,e) (* \a. e *) -> 
        let ty_e = type_check (("",TyVar a)::env) e in
        TyPoly(a,ty_e)
        (* You are monomorphizing a polymorphic expression e by instantiating it on type t *)
      | TyApp(e,t) ->
        begin
          match type_check env e with
          (* e: \'a. 'a -> 'a;
             e ['b] : ['b/'a] 'a -> 'a = ('b -> 'b)
           *)
          | TyPoly(a,typ) -> subst (t,a) typ
          | _ -> raise TypeError
        end
end

error: compile_error

In [26]:
let e = TyLam("'b", Lam("f", TyPoly("'a",TyArr(TyVar "'a", TyVar "'a")), 
                Lam("x", TyVar "'b", 
                        App(TyApp(Var "f", TyVar "'b"), Var "x")))) in
type_check [] e

- : typ =
TyPoly ("'b",
 TyArr (TyPoly ("'a", TyArr (TyVar "'a", TyVar "'a")),
  TyArr (TyVar "'b", TyVar "'b")))


```
(*
 * ∀'b. ((∀'a. 'a -> 'a) -> 'b -> 'b)
 *)
```
* Polymorphism we have in OCaml (or SML) is a parametric polymorphism
    * Why parameteric? Because we have ∀. We can instiate ∀-bound variable with any type, and we get a new type.
* What about polymorphism we have in Java?
    * `∀T. LinkedList<T>` instantiate T with Int --> `LinkedList<Int>`. We have parametric polymorphism.
    * `var l = new LinkedList<Animal>(); l.add(new Dog("Snoopy"))`. We also have Ad-hoc polymorphism. 
    * Java lets you combine both. `∀T. SortedList<T <: Comparable>`. 
* Parametric polymorphism can have multiple variants:
   * rank-1 polymorphism (OCaml/Haskell/ Java) vs higher-rank (System F, Coq, Isabelle) polymorphism
   * Predicative (OCaml/Haskell/ Java) vs Impredicative (System F, Coq, Isabell, F*)  polymorphism.
   
   
